In [1]:
%pip install torch --index-url https://download.pytorch.org/whl/cu130

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu130
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\joanc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\joanc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# Importar dependencias
import xgboost as xgb
import numpy as np
import shutil
import glob
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

import sys
import os

sys.path.append(os.path.abspath('..'))
from src.v2.data_loader import data_generator
from src.v2.maps import OTHER_LABEL

C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Paths de los archivos
csv_file = "../data/processed/dataset_reducido_30p.csv"
model_output = "../models/modeloxgb_vulnerabilidades.json"

In [5]:
val_dir = "../data/temp_validation"

if os.path.exists(val_dir):
    shutil.rmtree(val_dir)
os.makedirs(val_dir)

In [6]:
# Función principal de entrenamiento incremental
def train_incremental():

    # Parámetros de XGBoost
    params = {
        ## Probabilidades multiclase
        'objective': 'multi:softprob',
        ## Número de clases (0=Safe, 1..N=Top CWEs, N+1=Other)  
        'num_class': OTHER_LABEL + 1,
        ## Profundidad máxima de cada árbol
        'max_depth': 12,
        ## Tasa de aprendizaje (eta) 
        'learning_rate': 0.02,
        ## Método de construcción de árboles (hist: basado en histogramas)
        'tree_method': 'hist',
        ## Uso de GPU para entrenamiento
        'device': 'cuda',
        ## Métrica de evaluación (logaritmo de pérdida para clasificación multiclase)
        'eval_metric': 'mlogloss',
        ## Uso de todos los núcleos posibles
        'n_jobs': -1,
        ## Muestreo de columnas: Cada vez que crea un árbol, usa solo el 60% de las columnas al azar.
        'colsample_bytree': 0.6, 
        ## Muestreo de filas: Usar todas las filas por entrenamiento incremental
        'subsample': 1.0,
        ## Peso mínimo en hojas valores comunes [1, 3, 5]
        'min_child_weight': 3
    }
    
    model = None
    chunk_idx = 0
    
    # Iteramos sobre el generador
    for X_chunk, y_chunk in data_generator(csv_file, chunk_size=5000):
        chunk_idx += 1
        print(f"-------- Entrenando Chunk #{chunk_idx} (Tamaño: {len(y_chunk)}) --------")
        
        # Hacer un split interno para guardar las particiones para evaluar al final
        X_train, X_val, y_train, y_val = train_test_split(X_chunk, y_chunk, test_size=0.1, random_state=42)
        
        # Guardar datos de validación para evaluación posterior
        np.save(os.path.join(val_dir, f"X_val_{chunk_idx}.npy"), X_val)
        np.save(os.path.join(val_dir, f"y_val_{chunk_idx}.npy"), y_val)
        
        sample_weights = np.ones(len(y_train), dtype=float)
        
        # Supongamos tu nuevo mapa: 
        # 1: XSS, 2: SQLi, 3: CSRF, 4: Path, 5: OS Cmd, 6: File Up
        
        # Clase 1 (XSS): Recall decente, peso moderado.
        sample_weights[y_train == 1] = 3.0 
        
        # Clase 2 (SQLi): Tiene buena precisión, ¡EMPUJAR DURO para subir Recall!
        sample_weights[y_train == 2] = 20.0 
        
        # Clase 3, 5 (CSRF, OS Cmd): Peso alto estándar
        sample_weights[y_train == 3] = 10.0
        sample_weights[y_train == 5] = 10.0

        # Clase 4 (Path Traversal - El Paranoico): 
        # BAJAR PESO. Está gritando demasiado.
        sample_weights[y_train == 4] = 2.0 
        
        # Clase 6 (File Upload - La Joya): 
        # Precisión 94%. ¡EMPUJAR AL MÁXIMO! Podemos permitirnos más falsos positivos.
        sample_weights[y_train == 6] = 25.0
        
        # Convertir a DMatrix (formato optimizado de XGBoost)
        dtrain = xgb.DMatrix(X_train, label=y_train, weight=sample_weights)
        
        # ENTRENAMIENTO INCREMENTAL
        # xgb_model=model le dice que continúe desde el estado anterior
        # num_boost_round= añade N árboles nuevos por cada chunk
        model = xgb.train(
            params, 
            dtrain, 
            num_boost_round=50, 
            xgb_model=model
        )

        # Guardar checkpoint cada cierto tiempo
        if chunk_idx % 5 == 0:
            model.save_model(f"../models/checkpoints/checkpoint_{chunk_idx}.json")
            print(f"[INFO] Checkpoint guardado: checkpoint_{chunk_idx}.json")

    # Guardar modelo final
    model.save_model(model_output)
    print(f"[INFO] Entrenamiento finalizado. Modelo guardado en {model_output}")

In [ ]:
## Entrenar el modelo
train_incremental()

--- CodeBERT cargado en: cuda ---


C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

[INFO] Iniciando lectura de ../data/processed/dataset_reducido_30p.csv en chunks de 5000...
-------- Entrenando Chunk #1 (Tamaño: 5000) --------


In [ ]:
# Importar librerías de metricas
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def get_smart_predictions(probs_batch, threshold=0.15, safe_threshold=0.8):
    """
    Aplica lógica de umbrales para rescatar vulnerabilidades con baja confianza pero alto riesgo.
    """
    final_preds = []
    # Asumimos que la última columna es 'Other'. 
    # Si tienes 12 clases (0-11), el índice de Other es 11.
    other_idx = probs_batch.shape[1] - 1 
    
    for p in probs_batch:
        # 1. Regla de "Safe" (Clase 0)
        # Solo confiamos en Safe si la probabilidad es abrumadora (> 80%)
        if p[0] > safe_threshold:
            final_preds.append(0)
            continue
            
        # 2. Regla de Vulnerabilidades Específicas
        # Buscamos la vulnerabilidad más fuerte entre las columnas 1 y la penúltima (ignoramos Safe y Other)
        # p[1:other_idx] toma desde la clase 1 hasta la anterior a Other
        vuln_probs = p[1:other_idx] 
        
        # Si no hay columnas intermedias (ej. solo Safe y Other), saltamos
        if len(vuln_probs) > 0:
            best_vuln_local_idx = np.argmax(vuln_probs)
            best_vuln_prob = vuln_probs[best_vuln_local_idx]
            
            # El índice real es local_idx + 1 (porque nos saltamos la clase 0)
            real_vuln_idx = best_vuln_local_idx + 1
            
            # Si esa vulnerabilidad supera el umbral de riesgo (15%), ¡Alerta!
            if best_vuln_prob > threshold:
                final_preds.append(real_vuln_idx)
                continue
        
        # 3. Fallback (Si no es Safe fuerte, ni Vuln detectada)
        # Nos quedamos con el argmax clásico (probablemente sea Other o un Safe débil)
        final_preds.append(np.argmax(p))
        
    return np.array(final_preds)

In [ ]:
# Cargar el modelo entrenado
model_eval = xgb.Booster()
model_eval.load_model("../models/checkpoints/checkpoint_10.json")

# Usar GPU para predecir rápido
model_eval.set_param({"device": "cuda"})

# Listas
y_true_all = []
y_pred_all = []
y_pred_all_prob = [] # Lista de arrays

# Buscamos los archivos
val_files_X = sorted(glob.glob(os.path.join(val_dir, "X_val_*.npy")))
val_files_y = sorted(glob.glob(os.path.join(val_dir, "y_val_*.npy")))

print(f"Evaluando sobre {len(val_files_X)} chunks de validación...")

for f_x, f_y in zip(val_files_X, val_files_y):
    # Cargar solo este trocito en RAM
    X_val_part = np.load(f_x)
    y_val_part = np.load(f_y)
    
    # Predecir Probabilidades
    dtest = xgb.DMatrix(X_val_part)
    preds_prob = model_eval.predict(dtest)
    
    # Guardar probabilidades crudas (para analizar después por si acaso)
    y_pred_all_prob.append(preds_prob)
    
    preds_class = get_smart_predictions(preds_prob, threshold=0.15, safe_threshold=0.8)
    # -------------------------------------

    # Guardar resultados
    y_true_all.extend(y_val_part)
    y_pred_all.extend(preds_class)
    
    # Limpieza forzada
    del X_val_part, y_val_part, dtest, preds_prob

Evaluando sobre 11 chunks de validación...


In [67]:
# Reporte Final
print("\n--- Reporte de Clasificación (Smart Predict) ---")
print(classification_report(y_true_all, y_pred_all))

print("\nMatriz de Confusión:")
cm = confusion_matrix(y_true_all, y_pred_all)
print(cm)

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Label (Smart)')
plt.ylabel('True Label')
plt.title('Confusion Matrix con Thresholding (Riesgo > 15%)')
plt.show()


--- Reporte de Clasificación (Smart Predict) ---


KeyboardInterrupt: 